In [9]:
# 항상 기본으로 실행되는 도구

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

tensorflow 와 구체적으로 비교 시작

```python

# tensor flow 에서의 랜덤 시드 고정 ==> numpy에 의존하기 때문에

import numpy

np.randoms.seed(777)
```

In [3]:
# cuda 는 gpu 환경을 설정하는 경우만 사용하면 되므로 굳이 설정해줄 필요 없다
# random seed 설정

torch.manual_seed(777)

### tensor flow

```python

import tensorflow as tf

# 입력과 출력의 크기 정의
input_size = 2
output_size = 1

# 모델 정의 ==> nn.linear 와 같은 역할 (Dense)
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(1, input_shape=(2, ), activation='sigmoid')
])

# 입력 데이터와 레이블 정의
x_train = tf.constant([[0,0],[0,1],[1,0],[1,1]], dtype=tf.float32)
y_train = tf.constant([[0],[1],[1],[0]], dtype=tf.float32)

# 손실 함수와 최적화 함수 정의
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

# 학습
model.compile(optimizer=optimizer, loss=loss_fn)
model.fit(x_train, y_train, epochs=10000, verbose=0)

# 테스트
test_input = tf.constant([[0,0],[0,1],[1,0],[1,1]], dtype=tf.float32)
test_label = tf.constant([[0],[1],[1],[0]], dtype=tf.float32)
test_output = model.predict(test_input)
print('Input: \n', test_input.numpy())
print('Label: \n', test_label.numpy())
print('Output: \n', test_output.round(2))

```

### pytorch

In [14]:
# 입력데이터와 모델 정의

X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = torch.FloatTensor([[0], [1] , [1], [0]])

In [15]:
# 모델 정의

linear = nn.Linear(2, 1, bias = True)
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear, sigmoid)

In [16]:
print(model.parameters())

<generator object Module.parameters at 0x7f5ac078ba00>


In [17]:
# 손실함수와 최적화 함수의 정의

criterion = nn.BCELoss() #Binary Cross Entropy Loss
optimizer = optim.SGD(model.parameters(), lr = 1)

- 차이점

PyTorch에서는 fit 메소드와 같은 기능을 직접 구현해야 한다.

In [20]:
nb_epochs = 10001

for step in range(nb_epochs):
    optimizer.zero_grad()
    hypothesis = model(X)

    # 비용함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0: # 100번째 에포크마다 비용 출력
        print(step, cost.item())


## 단층 퍼셉트론으로는 해당 문제를 풀수 없기 때문에 ==> 특정수준부터 cost 함수의 값이 줄어들지 않는 현상이 발생한다.

0 0.6931471824645996
100 0.6931471824645996
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471824645996
700 0.6931471824645996
800 0.6931471824645996
900 0.6931471824645996
1000 0.6931471824645996
1100 0.6931471824645996
1200 0.6931471824645996
1300 0.6931471824645996
1400 0.6931471824645996
1500 0.6931471824645996
1600 0.6931471824645996
1700 0.6931471824645996
1800 0.6931471824645996
1900 0.6931471824645996
2000 0.6931471824645996
2100 0.6931471824645996
2200 0.6931471824645996
2300 0.6931471824645996
2400 0.6931471824645996
2500 0.6931471824645996
2600 0.6931471824645996
2700 0.6931471824645996
2800 0.6931471824645996
2900 0.6931471824645996
3000 0.6931471824645996
3100 0.6931471824645996
3200 0.6931471824645996
3300 0.6931471824645996
3400 0.6931471824645996
3500 0.6931471824645996
3600 0.6931471824645996
3700 0.6931471824645996
3800 0.6931471824645996
3900 0.6931471824645996
4000 0.6931471824645996
4100 0.6931471824645996
4200

_*with torch.no_grad():는 PyTorch에서 제공하는 context manager로, 이 내부에서는 
 gradient 계산을 수행하지 않는다는 것을 의미합니다._*

딥러닝에서는 모델을 학습할 때 역전파(backpropagation)를 통해 gradient를   계산하고, 이를 통해 모델의 가중치를 업데이트합니다. 이 과정은 많은 연산량을   필요로 하며, 계산한 gradient를 추적(track)하여 메모리에 저장하게 됩니다. \

그러나 모델을 평가하거나 예측을 수행할 때는 실제로 가중치를 업데이트하지 않기 
   때문에, gradient를 계산하거나 추적할 필요가 없습니다. torch.no_grad() 블록
   내에서 수행되는 연산들은 gradient 계산을 위한 추적을 비활성화하므로, 이를    통해 불필요한 메모리 사용을 줄일 수 있습니다. 따라서, torch.no_grad()는 주로
   모델 평가나 예측을 수행할 때 사용하게 됩니다.

In [22]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ')
    print(hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ')
    print(predicted.detach().cpu().numpy())
    print('실제값(Y): ')
    print(Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis): 
[[0.5]
 [0.5]
 [0.5]
 [0.5]]
모델의 예측값(Predicted): 
[[0.]
 [0.]
 [0.]
 [0.]]
실제값(Y): 
[[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  0.5
